In [3]:
from pathlib import Path
import pandas as pd 
import arrow

In [4]:
dataset_path = Path().absolute()

In [6]:
def read_df(in_filepath: Path) -> pd.DataFrame:

    return pd.read_csv(in_filepath)

In [27]:
flight_path = dataset_path / "flight_data"
weather_path = dataset_path / "weather-data"
output_path = dataset_path / "final-weather-flight-data"

In [15]:
flights = read_df(flight_path/"YUL-Departures-3months.csv")
weather = read_df(weather_path / "YUL-3months-Full-Weather.csv")

In [19]:
# can I write a function to do this? Yes. 
# writing it out to test if a vectorized operation is better than writing a function

flights['Scheduled Departure Time'] = flights['Scheduled Departure Time'].apply(
    lambda x: arrow.get(x.replace('t', 'T')).format('YYYY-MM-DD HH:00:00') if isinstance(x, str) else x)

flights['Estimated Departure Time'] = flights['Estimated Departure Time'].apply(
    lambda x: arrow.get(x.replace('t', 'T')).format('YYYY-MM-DD HH:00:00') if isinstance(x, str) else x)

flights['Actual Departure Time'] = flights['Actual Departure Time'].apply(
    lambda x: arrow.get(x.replace('t', 'T')).format('YYYY-MM-DD HH:00:00') if isinstance(x, str) else x)


In [21]:
flights['Scheduled Departure Time'] = flights['Scheduled Departure Time'].apply(
    lambda x: arrow.get(x).format("YYYY-MM-DD HH:00:00") if pd.notna(x) else x)

flights['Estimated Departure Time'] = flights['Estimated Departure Time'].apply(
    lambda x: arrow.get(x).format("YYYY-MM-DD HH:00:00") if pd.notna(x) else x)

flights['Actual Departure Time'] = flights['Actual Departure Time'].apply(
    lambda x: arrow.get(x).format("YYYY-MM-DD HH:00:00") if pd.notna(x) else x)

In [22]:
flights.head(5)

,Type,Status,Departure IATA Code,Departure Gate,Departure Delay (min),Scheduled Departure Time,Estimated Departure Time,Actual Departure Time,Arrival IATA Code,Arrival Terminal,Scheduled Arrival Time,Estimated Arrival Time,Airline Name,Flight Number,IATA Flight Number
0,departure,active,yul,a61,23,2024-01-12 19:00:00,2024-01-12 19:00:00,2024-01-12 19:00:00,lhr,2,2024-01-13t06:30:00.000,2024-01-13t06:11:00.000,aegean airlines,3089,a33089
1,departure,active,yul,a63,33,2024-01-12 19:00:00,2024-01-12 19:00:00,2024-01-12 19:00:00,cdg,2e,2024-01-13t08:05:00.000,2024-01-13t07:45:00.000,delta air lines,8612,dl8612
2,departure,active,yul,a57,0,2024-01-12 19:00:00,2024-01-12 19:00:00,NaN,ams,NaN,2024-01-13t08:05:00.000,NaN,air france,8276,af8276
3,departure,active,yul,a49,115,2024-01-12 19:00:00,2024-01-12 21:00:00,NaN,yyz,1,2024-01-12t20:46:00.000,NaN,azul,7807,ad7807
4,departure,active,yul,a6,30,2024-01-12 19:00:00,2024-01-12 19:00:00,NaN,yyz,3,2024-01-12t21:00:00.000,NaN,porter,130,p3130


In [24]:
flights_and_weather = flights.merge(weather, how = "left", left_on = "Scheduled Departure Time", right_on = "Timestamp")

In [28]:
flights_and_weather.to_csv(output_path / "dataset_YUL-Flights-Weather.csv", index = False)